In [3]:
import numpy as np

# 0 -> head 1 -> tail
def toss_coin(times):
    return np.random.choice(2,times)

def toss_experiment(times):
    return list(map(lambda x: [x,toss_coin(10)],range(times)))

In [419]:
total_freq_min = 0
total_freq_rand = 0
total_freq_first = 0
for i in range(1000):
    experiment = toss_experiment(1000)
    first = experiment[0]
    rand = experiment[np.random.choice(len(experiment)) - 1]

    min_heads = max(experiment,key = lambda x: sum(x[1]))

    total_freq_min += (10 - sum(min_heads[1]))/10
    total_freq_first += (10 - sum(first[1]))/10
    total_freq_rand += (10 - sum(rand[1]))/10

print('min freq :',total_freq_min/10000)    
print('first freq :',total_freq_min/10000)    
print('rand freq :',total_freq_min/10000)    

min freq : 0.00372
first freq : 0.00372
rand freq : 0.00372


In [4]:
def generateLine():
    x1,y1 = np.random.uniform(-1,1),np.random.uniform(-1,1)
    
    x2,y2 = np.random.uniform(-1,1),np.random.uniform(-1,1)
    eq = np.array([[x1,1],[x2,1]])
    res = np.array([y1,y2])
    return np.linalg.solve(eq,res)

def classify(slope,intercept,x,y):
    w = (y - intercept)/float(slope)
    if(x > w):
        return 1
    else:
        return -1
    

In [11]:
import numpy.linalg as linalg
from functools import partial

in_freq=[]
out_freq=[]

SAMPLE_SIZE = 100
EXPERIMENT_TIMES = 1
OUT_OF_SAMPLE_SIZE = 1000

slope,intercept = generateLine()
f = partial(classify,slope,intercept) 
  
def train_linear(features,labels):
    return linalg.pinv(features).dot(labels)

def predict(user_features,optimal_weights):
    return np.sign(np.array(user_features).dot(optimal_weights))
        
        
for i in range(EXPERIMENT_TIMES): 
    features = [[np.random.uniform(-1,1),np.random.uniform(1,-1)] for i in range(SAMPLE_SIZE)]
    out_of_sample = [[np.random.uniform(-1,1),np.random.uniform(1,-1)] for i in range(OUT_OF_SAMPLE_SIZE)]
    
    labels = list(map(lambda x: f(x[0],x[1]),features))
    out_of_sample_labels = list(map(lambda x: f(x[0],x[1]),out_of_sample))
  
    optimal_weights = np.array(train_linear(np.matrix(features),labels))[0]
    
    in_error = 0
    for i in range(len(features)):
        user_weights = features[i]
        expected = labels[i]
        actual = predict(user_weights,optimal_weights)
        if(actual != expected):   
            in_error += 1
    in_freq.append(in_error/float(SAMPLE_SIZE))
    
    out_error=0
    for i in range(len(out_of_sample_labels)):
        user_weights = out_of_sample[i]
        expected = out_of_sample_labels[i]
        actual = predict(user_weights,optimal_weights)
        if(actual != expected):   
            out_error += 1
            
    out_freq.append(out_error/float(OUT_OF_SAMPLE_SIZE))

print(np.average(in_freq)) #answer to the 5th -> c

print(np.average(out_freq)) #answer to the 6th -> c

0.14
0.07


In [22]:
SAMPLE_SIZE = 10
EXPERIMENT_SIZE = 1000
features = [[np.random.uniform(-1,1),np.random.uniform(1,-1)] for i in range(SAMPLE_SIZE)]
labels = list(map(lambda x: f(x[0],x[1]),features))

weights = train_linear(features,labels)
print(weights)

def train_PLA(weights,features,labels,learning_rate):
    iterations = 0
    minimum_error = -1
    best_weights = []
    error_sum = 1
    while(error_sum != 0):
        error_sum = 0
        iterations += 1
        for i in range(len(features)):
            row = features[i]
            expected = labels[i]
            actual = predict(row,weights)
            if actual != expected:
                error_sum += 1
                for w in range(len(weights) -1):
                    weights[w + 1] = weights[w + 1] + learning_rate * row[w + 1]
    
        if(minimum_error == -1 or error_sum < minimum_error):
            minimum_error = error_sum
            best_weights = list(weights) 
                
    return best_weights,iterations

iterations_vec = []

for i in range(EXPERIMENT_SIZE):    
    weights,iterations = train_PLA(weights,features,labels,0.1)
    iterations_vec.append(iterations)

print(np.average(iterations_vec)) #answer to 7th --> a

[ 1.56419445  0.72173249]
1.0


In [80]:
SAMPLE_SIZE = 1000
EXPERIMENT_SIZE = 1000

in_freq =  []

def non_linear_target(x1,x2):
    return np.sign((x1**2)+(x2**2)-0.6)
    
for j in range(EXPERIMENT_SIZE):
    features = [[np.random.uniform(-1,1),np.random.uniform(1,-1)] for i in range(SAMPLE_SIZE)]
    
    features_and_labels = list(map(lambda x: (x,non_linear_target(x[0],x[1])),features))
    
    np.random.shuffle(features_and_labels)
    
    for i in (range(int(SAMPLE_SIZE/10))):
        features_and_labels[i] = (features_and_labels[i][0],-1*features_and_labels[i][1])
    
    random_noise_features = list(map(lambda x: x[0],features_and_labels))
    random_noise_labels = list(map(lambda x: x[1],features_and_labels))
    
    optimal_weights = np.array(train_linear(np.matrix(random_noise_features),random_noise_labels))[0]
    
    in_error = 0
    for i in range(SAMPLE_SIZE):
        user_weights = random_noise_features[i]
        expected = random_noise_labels[i]
        actual = predict(user_weights,optimal_weights)
        
        if(actual != expected):   
            in_error += 1
    in_freq.append(in_error/float(SAMPLE_SIZE))

print(np.average(in_freq))   # answer to 8th , WRONG, get avg of 0.17, correct answer is 0.5 
                             # I've fix it !!! LOL LOL (I forgot to use the nonlinear target function...)


0.482809


In [107]:
SAMPLE_SIZE = 1000
EXPERIMENT_SIZE = 1000

in_freq =  []

def non_linear_target(x1,x2):
    return np.sign((x1**2)+(x2**2)-0.6)

def non_linear_transformation(x1,x2):
    return [1,x1,x2,x1*x2,x1**2,x2**2]

def transform_features(features):
    res = []
    for i in range(len(features)):
        res.append(non_linear_transformation(features[i][0],features[i][1]))
    return res

crazy_weights = []
for j in range(EXPERIMENT_SIZE):
    features = [[np.random.uniform(-1,1),np.random.uniform(1,-1)] for i in range(SAMPLE_SIZE)]
    
    features_and_labels = list(map(lambda x: (x,non_linear_target(x[0],x[1])),features))
    
    np.random.shuffle(features_and_labels)
    
    for i in (range(int(SAMPLE_SIZE/10))):
        features_and_labels[i] = (features_and_labels[i][0],-1*features_and_labels[i][1])
    
    random_noise_features = transform_features(list(map(lambda x: x[0],features_and_labels)))
    random_noise_labels = list(map(lambda x: x[1],features_and_labels))
    
    optimal_weights = np.array(train_linear(np.matrix(random_noise_features),random_noise_labels))[0]
    crazy_weights.append(optimal_weights)

    in_error = 0
    for i in range(SAMPLE_SIZE):
        user_features = transform_features([random_noise_features[i]])[0]
        expected = random_noise_labels[i]
        actual = predict(random_noise_features[i],optimal_weights)
        
        if(actual != expected):  
            in_error += 1
    in_freq.append(in_error/float(SAMPLE_SIZE))

print(np.average(in_freq))  #answer to 10th exercise blaaah

#So... I am not sure WTF but, in order to get a good prediction result I CANT 
#transform the user_weights with the transfome_features???? why?? 
#I thought that when we use nonlinear transformation we had to transform the user_features with the same fn we
#picked to train our data... HELP

def crazy_fn(n):
    return np.average(list(map(lambda x : x[n],crazy_weights)))

print("x1:",crazy_fn(1))    
print("x2:",crazy_fn(2))    
print("x1*x2:",crazy_fn(3))    
print("x1**2:",crazy_fn(4))    
print("x2**2:",crazy_fn(5))    #all this results combine gave me the answer A in the 9th question, and it's correct...
#NOT BAD :not_bad:


0.12386
x1: -0.000318529445794
x2: -0.000450648393481
x1*x2: -0.000101106847093
x1**2: 1.56273727711
x2**2: 1.56082115004


5.0